## Read PRSIM

calculate precipitation data for Midwest region (box)

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import dask.array as da
import glob as glob
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

In [2]:
import geopandas as gpd
import regionmask
import cartopy.io.shapereader as sr

In [3]:
from project_utils import parameters as param

#### processed data files created in this notebook:

In [4]:
region_shp_file = "../processed_data/region.shp"
pr_mean_file = "../processed_data/prism_mean_"+str(param.time_period.start)+"_"+str(param.time_period.stop)+".nc"
region_mean_file = "../processed_data/region_mean_precip.csv"

#### Read PRISM data:

In [5]:
prism_path = "../input_data/PRISM/"
pr_prism_files = sorted(glob.glob(prism_path+"*.nc"))

In [6]:
pr_prism = xr.open_mfdataset(pr_prism_files, combine='nested', concat_dim='time').sel(
    lat = param.region_lat, lon = param.region_lon)

pr_prism['time'] = pd.to_datetime(pr_prism['time'].values, format='%Y%m%d')

#### Create Midwest region shapefile:

In [7]:
from shapely.geometry import Polygon

In [8]:
rx = np.interp(np.arange(0,100), [0, 25, 50, 75, 100], param.region_box_x) ## x coordinates
ry = np.interp(np.arange(0,100), [0, 25, 50, 75, 100], param.region_box_y) ## y coordinates

In [9]:
region_shp = gpd.GeoDataFrame(index=[0], crs='epsg:4326', 
                          geometry=[Polygon(zip(rx, ry))])

In [10]:
region_shp.to_file(filename=region_shp_file, driver="ESRI Shapefile")

In [11]:
region_mask = regionmask.mask_geopandas(region_shp, pr_prism)

#### Calculate mean precipitation (grid-level):

In [12]:
calc_climatology = True
## set to false to read already calculated data

In [14]:
if(calc_climatology):
    pr_mean = pr_prism.sel(time = param.time_period).mean(dim = "time")
    pr_mean.to_netcdf(pr_mean_file)

pr_mean = xr.open_dataset(pr_mean_file)

/oak/stanford/groups/omramom/group_members/fvdav/projects/precip_clustering/envsnew/lib/python3.6/site-packages/dask/array/numpy_compat.py:40: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


#### Regional precipitation time-series

In [15]:
region_mean = pr_prism.sel(lon = param.region_lon, lat = param.region_lat).mean(dim = ['lat', 'lon'])
region_mean = region_mean.to_dataframe()

In [16]:
region_mean.to_csv(region_mean_file)